## Part 1: Neural Network from Scratch

1. Importing basic libraries

In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, average_precision_score, confusion_matrix, classification_report, f1_score

2. Neural Network

In [ ]:
class NeuralNetwork:
    def __init__(self, layer_dim, seed=42):
        np.random.seed(seed)
        self.layer_dim = layer_dim
        self.parameters = {}
        self.L = len(layer_dim)-1
        for i in range(1,len(layer_dim)):
            self.parameters[f"W{i}"] = np.random.randn(layer_dim[i],layer_dim[i-1])*np.sqrt(2./layer_dim[i-1])
            self.parameters[f"b{i}"] = np.zeros((layer_dim[i],1))

    def relu(self,z):
        return np.maximum(0,z)

    def sigmoid(self,z):
        return 1/(1+np.exp(-z))

    def forward(self,X):
        A = X.T
        cache = {"A0":A}
        for i in range(1,self.L+1):
            W = self.parameters[f"W{i}"]
            b = self.parameters[f"b{i}"]
            Z = W @ A + b
            A = self.relu(Z) if i < self.L else self.sigmoid(Z)
            cache[f"Z{i}"]=Z
            cache[f"A{i}"]=A
        return A.T, cache


    def compute_loss(self, Y_hat, Y):
        m = Y.shape[0]
        epsilon = 1e-8
        loss = -np.mean(Y * np.log(Y_hat + epsilon) + (1 - Y) * np.log(1 - Y_hat + epsilon))
        return loss

    def relu_backward(self, dA, Z):
        dZ = np.array(dA, copy=True)
        dZ[Z<=0]=0
        return dZ

    def sigmoid_backward(self, dA, Z):
        s = self.sigmoid(Z)
        return dA*s*(1-s)

    def backward(self, X, Y, cache):
        grads = {}
        m = X.shape[0]
        L = self.L
        A_prev = cache[f"A{L-1}"]
        ZL = cache[f"Z{L}"]
        AL = self.sigmoid(ZL)
        dZL = AL - Y.T
        grads[f"dW{L}"] = (1/m) * np.dot(dZL, A_prev.T)
        grads[f"db{L}"] = (1/m) * np.sum(dZL, axis=1, keepdims=True)
        dA_prev = np.dot(self.parameters[f"W{L}"].T, dZL)
        for i in reversed(range(1, L)):
            Z = cache[f"Z{i}"]
            A_prev = cache[f"A{i-1}"]
            dZ = self.relu_backward(dA_prev, Z)
            grads[f"dW{i}"] = (1/m) * np.dot(dZ, A_prev.T)
            grads[f"db{i}"] = (1/m) * np.sum(dZ, axis=1, keepdims=True)
            dA_prev = np.dot(self.parameters[f"W{i}"].T, dZ)
        return grads

    def update_params(self, grads, learning_rate):
        for l in range(1, self.L+1):
            self.parameters[f"W{l}"]-=learning_rate*grads[f"dW{l}"]
            self.parameters[f"b{l}"]-=learning_rate*grads[f"db{l}"]

    def train(self, X, Y, epochs=100, learning_rate=0.01, verbose=True):
        for i in range(epochs):
            Y_hat, cache = self.forward(X)
            loss = self.compute_loss(Y_hat, Y)
            grads = self.backward(X, Y, cache)
            self.update_params(grads, learning_rate)
            if verbose and i % 10 == 0:
                print(f"Epoch {i}, Loss: {loss:.4f}")
                
    def predict(self, X, threshold=0.5):
        Y_hat, _ = self.forward(X)
        return (Y_hat > threshold).astype(int)


2. Importing Data as a Pandas Dataframe

In [ ]:
df = pd.read_csv('data.csv')
X = df.drop(columns=['no_show_bin']).astype(np.float32).values
y = df['no_show_bin'].astype(np.float32).values.reshape(-1, 1)

3. Scaling numerical features

In [6]:
continuous_cols = ['age', 'wait_days', 'sched_hour','past_appointments', 'patient_no_show_rate']
target_col = 'no_show_bin'
binary_cols = [col for col in df.columns if col not in continuous_cols + [target_col]]
X_continuous = df[continuous_cols]
X_binary = df[binary_cols].astype(np.float32)
y = df[target_col].astype(np.float32).values.reshape(-1, 1)
scaler = StandardScaler()
X_continuous_scaled = scaler.fit_transform(X_continuous)
X = np.hstack([X_continuous_scaled, X_binary.values])

4. Train Test Split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

5. Training

In [10]:
n = X_train.shape[1]
layer_dim = [n,32,16,1]

nn = NeuralNetwork(layer_dim,seed=42)
start = time.time()
nn.train(X_train,y_train,epochs=200,learning_rate=0.01,verbose=False)
end = time.time()

6. Evaluation on test data

In [ ]:
y_pred = nn.predict(X_test,threshold=0.5)

print("Training time:",end-start)
print("Test Accuracy:", accuracy_score(y_test, y_pred))
print("F1-Score:",f1_score(y_test, y_pred))
print("PR-AUC: ",average_precision_score(y_test,y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Training time: 34.58730936050415
Test Accuracy: 0.8818313427433948
F1-Score: 0.6176814988290398
PR-AUC:  0.527539415842822
Confusion Matrix:
 [[17382   259]
 [ 2353  2110]]

Classification Report:
               precision    recall  f1-score   support

         0.0       0.88      0.99      0.93     17641
         1.0       0.89      0.47      0.62      4463

    accuracy                           0.88     22104
   macro avg       0.89      0.73      0.77     22104
weighted avg       0.88      0.88      0.87     22104

